# Ch05. Chains- 여러 프로세스를 통합
Chains는 일련의 처리를 하나의 묶음처럼 처리할 수 잇는 모듈.
여기서는 3가지로 나눠서 소개함
- 1. 다중 처리를 할 수 있다.
- 2. 여러모듈을 쉽게 조합할 수 있는 Chains
- 3. 특정 기능에 특화된 Chains
- 4. Chains 자체 정리하기

## 1. 다중 처리를 할 수 있다.
> Chains의 종류에 대해 자세히 알아본다
1. 여러 모듈의 조합을 활용할 수 있다. 
- 여러개의 모듈을 조합해 하나로 묶을때 사용됨. LLMChain과 ConversationChain
2. 특정 용도에 특화된 Chain을 만들 수 있다.
- 언어 모델의 호출만으로는 대응하기 어려운 처리 (예> 에이전트 활용한 외부기능 연동 구현)를 앞/뒤에 넣어서 구현 가능
3. 체인자체를 정리한다.
- Chains는 기능덩어리이다. 이 기능덩어리들을 여러새 준비하여 순서대로 실행하거나 필요에 따라 호출할 수 있도록 Chains를 묶을 수 있다

여러개의 모듈을 조합해 하나로 묶을때 사용됨. LLMChain과 ConversationChain

## 2. 여러 모듈을 쉽게 조합할 수 있는 Chains
> 랭체인 내의 모듈을 결합하여 하나의 기능으로 만들기

<img src="./img/lang_5_1.png" width="70%" height="70%">


In [4]:
'''
PromptTemplate,ChatOpenAI를 결합하여 하나의 chain 만들기 
'''
from langchain import LLMChain, PromptTemplate  #← LLMChain 가져오기
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import openai
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()
os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(  
    model="gpt-3.5-turbo")

In [49]:
prompt = PromptTemplate(  
    template="{product}는 어느 회사에서 개발한 제품인가요?",  
    input_variables=["product"])

chain = LLMChain( #← LLMChain을 생성
    llm=chat,
    prompt=prompt,
    verbose=True) #← verbose로 답변 과정을 터미널에 표시할 수 있음

result = chain.predict(product="iPhone") #← LLMChain을 실행

print(result)



> Entering new LLMChain chain...
Prompt after formatting:
iPhone는 어느 회사에서 개발한 제품인가요?

> Finished chain.
아이폰은 애플이 개발한 제품입니다.


In [50]:
'''
ConversationChain으로 llm+memory활용
'''
from langchain import PromptTemplate, ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(model="gpt-3.5-turbo")
memory = ConversationBufferMemory(return_messages=True)
chain = ConversationChain(
    memory=memory,
    llm=chat,
    verbose=True
)

result = chain("iphone의 가격은?")
result



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: iphone의 가격은?
AI:

> Finished chain.


{'input': 'iphone의 가격은?',
 'history': [HumanMessage(content='iphone의 가격은?'),
  AIMessage(content='물론이죠! 현재 iPhone의 가격은 모델 및 저장 용량에 따라 다릅니다. 예를 들어, iPhone 13 Pro Max의 128GB 모델은 약 1,299달러이고, 256GB 모델은 약 1,399달러입니다. 또한, iPhone SE의 64GB 모델은 약 399달러이고, 128GB 모델은 약 449달러입니다. 다양한 모델과 옵션으로 가격이 다양하니 참고하세요!')],
 'response': '물론이죠! 현재 iPhone의 가격은 모델 및 저장 용량에 따라 다릅니다. 예를 들어, iPhone 13 Pro Max의 128GB 모델은 약 1,299달러이고, 256GB 모델은 약 1,399달러입니다. 또한, iPhone SE의 64GB 모델은 약 399달러이고, 128GB 모델은 약 449달러입니다. 다양한 모델과 옵션으로 가격이 다양하니 참고하세요!'}

> LLMChain은 내부에서 이뤄지는 처리가 적어 이해가 쉽지만, 프롬프트 생성 및 언어 모델 호출을 여러번 반복하는 Chains도 존재할 수 있기 때문에, 개발 시 verbose=True를 통해 내부에서 일어나는 처리를 추적할 수 있어야 한다

## 3. 특정 기능에 특화된 Chains
> 언어모델은 학습된 지식 외의 정보 기반으로 답변을 할 수 없다. 따라서, LLMRequestChain을 사용하면 외부롤 연결하는 Agent를 함께 구성할 수 있다

<img src="./img/lang_5_2.png" width="70%" height="70%">


In [70]:
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic

load_dotenv()

chat = ChatAnthropic(
    model="claude-3-haiku-20240307"
)
prompt = PromptTemplate( #← PromptTemplate을 초기화
    input_variables=["query",
                     "requests_result"],#←url의 requests 결과 가져옴
    template="""아래 참고문서를 바탕으로 질문에 답해 주세요.
참고문서: {requests_result}
질문: {query}""",
)

llm_chain = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
)

chain = LLMRequestsChain(  #← LLMRequestsChain을 초기화
    llm_chain=llm_chain,  #← llm_chain에 LLMChain을 지정
)

print(chain({
    "query": "도쿄의 날씨를 알려주세요",
    "url": "https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json",
}))



> Entering new LLMChain chain...
Prompt after formatting:
아래 참고문서를 바탕으로 질문에 답해 주세요.
참고문서: {"publishingOffice":"気象庁","reportDatetime":"2024-05-12T16:35:00+09:00","targetArea":"東京都","headlineText":"","text":"　サハリン付近の低気圧から前線が日本海へのびています。\n\n　東京地方は、曇りとなっています。\n\n　１２日は、湿った空気の影響を受ける見込みです。このため、曇りで、夜は雨の降る所があるでしょう。\n\n　１３日は、前線が東日本を通過する見込みです。このため、雨で、明け方から夕方は雷を伴って激しく降る所があるでしょう。伊豆諸島では雨で雷を伴い激しく降る所がある見込みです。\n\n【関東甲信地方】\n　関東甲信地方は、曇りで雨の降っている所があります。\n\n　１２日は、湿った空気の影響を受ける見込みです。このため、曇りで、雨の降る所があるでしょう。\n\n　１３日は、前線が東日本を通過する見込みです。このため、雨や曇りで、雷を伴い激しく降る所があるでしょう。\n\n　関東地方と伊豆諸島の海上では、１２日は波が高く、１３日はうねりを伴ってしける見込みです。また、所々で霧が発生するでしょう。船舶は高波に警戒し、視程障害に注意してください。"}
질문: 도쿄의 날씨를 알려주세요

> Finished chain.
{'query': '도쿄의 날씨를 알려주세요', 'url': 'https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json', 'output': '참고문서에 따르면 도쿄의 날씨는 다음과 같습니다:\n\n- 12일은 흐린 날씨가 예상되며, 밤에 비가 내릴 것으로 보입니다.\n- 13일은 전선이 동일본을 통과할 것으로 예상되어, 비가 오고 아침부터 저녁까지 천둥을 동반한 강한 비가 내릴 것으로 보입니다. \n- 이즈 제도에서도 비와 천둥을 동반한 강한 비가 있을 것으로 예상됩니다.\n\n즉, 도쿄는 12일 흐린 날씨

## 4. Chains 자체 정리하기
> SimpleSequentialChain은 여러개의 Chains을 하나로 묶어서 어려운 작업도 쉽게 구현

<img src="./img/lang_5_3.png" width="70%" height="70%">


In [78]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, AIMessagePromptTemplate
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()
os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(model="gpt-3.5-turbo")

write_article_chain = LLMChain( #← 기사를 쓰는 LLMChain 만들기
    llm=chat,
    prompt=PromptTemplate(
        template="{input}에 관한 기사를 써주세요.",
        input_variables=["input"],
    ),
)

translate_chain = LLMChain( #← 번역하는 LLMChain
    llm=chat,
    prompt=PromptTemplate(
        template="다음 문장을 영어로 번역해 주세요.\n{input}",
        input_variables=["input"],
    ),
)

email_chain = LLMChain( #← 이메일을 쓰는 LLMChain
    llm=chat,
    prompt=PromptTemplate(
        template="고객에게 이메일로 보낼 수 있도록 바꿔주세요 \n{input}",
        input_variables=["input"],
    ),
)

sequential_chain = SimpleSequentialChain( #← SimpleSequentialChain을 생성
    chains=[ #← 실행할 chains을 리스트로 한번에 담는다
        write_article_chain, 
        translate_chain,
        email_chain
    ],
    verbose=True
)

result = sequential_chain.run("일렉트릭 기타 선택 방법")

print(result)




> Entering new SimpleSequentialChain chain...
일렉트릭 기타를 선택하는 것은 중요한 결정이며, 많은 고려 사항이 필요합니다. 이 기사에서는 일렉트릭 기타를 선택하는 방법에 대해 알아보겠습니다.

1. 예산 설정: 먼저 예산을 설정해야합니다. 일렉트릭 기타는 다양한 가격대에 있으며, 예산에 맞는 기타를 선택하는 것이 중요합니다.

2. 취향과 스타일: 다양한 모양, 크기, 색상, 그리고 디자인의 일렉트릭 기타가 있습니다. 자신의 취향과 스타일에 맞는 기타를 선택하는 것이 중요합니다.

3. 음질과 사운드: 음질은 일렉트릭 기타를 선택할 때 가장 중요한 요소 중 하나입니다. 자신이 원하는 사운드와 음질을 고려하여 기타를 선택해야합니다.

4. 브랜드와 모델: 다양한 일렉트릭 기타 브랜드가 있으며, 각 브랜드마다 특징이 다릅니다. 자신에게 맞는 브랜드와 모델을 선택하는 것이 중요합니다.

5. 픽업 타입: 일렉트릭 기타에는 싱글 코일 픽업과 험버커커 픽업이 있습니다. 각각의 픽업은 다른 사운드를 제공하며, 자신이 원하는 사운드에 맞춰 선택해야합니다.

6. 악세서리와 장비: 일렉트릭 기타를 선택할 때는 악세서리와 장비도 함께 고려해야합니다. 앰프, 이펙터, 케이스 등을 함께 선택하여 전체적인 장비를 고려해야합니다.

이러한 요소들을 고려하여 자신에게 가장 적합한 일렉트릭 기타를 선택할 수 있을 것입니다. 적절한 기타를 선택하여 음악을 더욱 재미있게 즐기시길 바랍니다.
Choosing an electric guitar is an important decision and requires careful consideration. In this article, we will explore how to choose an electric guitar.

1. Budget Setting: First, you need to set a budget. Electric guitars come in various price ranges, so it

- 용도 별로 다양하게 준비된 Chains
1. RouterChain
> 미리 준비된 여러 Chains와 그 설명을 준비해두고, 질문이나 지시에 따라 어떤 Chains를 실행할지 판단한 후에 Chains를 실행한다. 이렇게 여러 Chains를 모아서 결과적으로 다양한 종류의 task를 하나의 Chains로 처리할 수 있게된다.
2. LLMMathChain
> 언어모델이 틀리기 쉬운 계산을 확실하게 수행하는 Chain. 언어모델에 파이썬코드를 실행하게 하여 확실한 계산을 하게함
3. LLMCheckerChain
> 입력한 프롬프트로 언어모델을 호출하고, 결과를 다시 언어모델을 호출해 검증함으로써 잘못된 결과를 출력하기 어렵게 만듬
4. OpenAIModerationChain
> 생성된 콘텐츠가 OpenAI의 정책에 준수하는지를 확인하는 Moderation기능. 폭력이나 차별, 자해등 문제가 있는 콘텐츠의 생성을 방지

## 5. 조건문처럼 쓸수 있는 RouterChain
> Router는 네트워크 중계기로, 여러 통신망중 최적의 경로로 지정해줌. RouterChain 은 질문의 종류에 따라 최적의 chain으로 연결해주는 장치

<img src="./img/lang_5_4.png" width="70%" height="70%">


- destination_chains: RouterChain 다음 단계로 연결될 수 있는 목적지 Chain들로, 각각의 destination chain은 LLMChain이나 다른 복합적인 Chain으로 생성
- default_chain: 입력 query를 보낼만한 적합한 destination chain이 없을 때 연결하는 Chain
- RouterChain: destination_chains 정보와 입력 query를 비교하여 가장 적합한 Chain을 선택. 적절한 Chain이 없으면 default
- RouterOutputParser: RouterChain의 출력 문자열을 다음 Chain에서 입력으로 사용할 수 있도록 딕셔너리로 변경. 딕셔너리는 목적지 Chain 정보와 목적지 Chain의 입력 프롬프트 정보를 포함

In [118]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,PromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

# 각 역할 생성
economic_expert = """
당신은 아담스미스입니다. 경제학 전문가입니다. 모르는 질문에는 모른다고 답합니다.
질문: {input}
"""
physics_expert = """
당신은 아인슈타인입니다. 물리학 전문가입니다. 모르는 질문에는 모른다고 답합니다.
질문: {input}
"""
IT_expert = """
당신은 30년 경력의 개발자입니다. 코딩, 네트워크, 인프라 등 IT와 컴퓨터에 관련한 전문가입니다. 모르는 질문에는 모른다고 답합니다.
질문: {input}
"""
philosophy_expert = """
당신은 쇼펜하우어입니다. 철학 전문가입니다. 모르는 질문에는 모른다고 답합니다.
질문: {input}
"""
# 각 Agent 생성
prompt_info = [
    {
        "name":"economic_expert",
        "description": "경제 관련 질문에 대답",
        "prompt_template": economic_expert
    },
    {
        "name":"physics_expert",
        "description": "물리학 관련 질문에 대답",
        "prompt_template": physics_expert
    },
    {
        "name":"IT_expert",
        "description": "컴퓨터 관련 질문에 대답",
        "prompt_template": IT_expert
    },
    {
        "name":"philosophy_expert",
        "description": "철학 관련 질문에 대답",
        "prompt_template": philosophy_expert
    }
]

llm = ChatOpenAI(temperature=0)

destination_chains = {} #←destination 딕셔너리 생성
for p in prompt_info:
    name = p["name"]
    prompt = ChatPromptTemplate.from_template(template=p["prompt_template"])
    destination_chains[name] = LLMChain(llm=llm, prompt=prompt)

destinations = [f"{p['name']}: {p['description']}" for p in prompt_info]
destinations_str = "\n".join(destinations)
print(destinations_str)

# default_chain 생성
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

MULTI_PROMPT_ROUTER_TEMPLATE = """
입력받은 내용 기반, 가장 적절한 모델 프롬프트 선택.
모델 프롬프트 정보는 다음과 같음.

"프롬프트 이름": "프롬프트 설명"

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination"은 아래 주어진 프롬프트 설명을 바탕으로 프롬프트 이름 중 하나를 선택하거나,
적절한 프롬프트가 없으면 "DEFAULT"를 선택.

REMEMBER: "next_inputs"은 원본 INPUT 입력.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain,
                         verbose=True
                        )


# chain.run("고민을 없애려면?")


economic_expert: 경제 관련 질문에 대답
physics_expert: 물리학 관련 질문에 대답
IT_expert: 컴퓨터 관련 질문에 대답
philosophy_expert: 철학 관련 질문에 대답


In [101]:
chain.run("코딩 공부는 어떻게 시작하는게 좋아?")



> Entering new MultiPromptChain chain...
IT_expert: {'input': '코딩 공부는 어떻게 시작하는게 좋아?'}
> Finished chain.


'코딩 공부를 시작하는 가장 좋은 방법은 기초적인 프로그래밍 언어부터 배우는 것입니다. 예를 들어 Python이나 Java와 같은 언어를 공부해보세요. 온라인 강의나 책을 통해 기본 문법과 개념을 익히고, 실제로 코딩을 해보면서 연습하는 것이 중요합니다. 또한 프로그래밍에 대한 관심과 열정을 가지고 지속적으로 학습하는 것이 중요합니다. 부족한 부분이 있으면 인터넷을 통해 검색하거나 도움을 요청하는 것도 좋은 방법입니다. 기초를 탄탄히 다지고 나서 원하는 분야나 프로젝트에 맞게 심화 학습을 진행하면 됩니다.'

In [117]:
chain.run("시간과 공간의 관계는?")



> Entering new MultiPromptChain chain...
physics_expert: {'input': '시간과 공간의 관계는?'}
> Finished chain.


'저는 물리학 전문가인 아인슈타인이지만, 시간과 공간의 관계에 대해서는 정확히 알려드릴 수 없습니다. 이는 상대성 이론과 관련된 매우 복잡하고 깊은 주제이기 때문입니다. 이에 대한 자세한 설명은 전문가에게 문의하시거나 관련 서적을 참고하시는 것이 좋습니다.'

In [119]:
chain.run("금리가 오르면 채권가격이 떨어지는 이유는?")



> Entering new MultiPromptChain chain...
economic_expert: {'input': '금리가 오르면 채권가격이 떨어지는 이유는?'}
> Finished chain.


'금리가 오르면 채권가격이 떨어지는 이유는 주로 이자율과 채권의 가격은 역의 관계에 있기 때문입니다. 채권의 이자율이 오르면 채권의 가격은 하락하게 됩니다. 이는 채권을 발행한 기업이나 정부가 채권을 발행할 때의 이자율이 높아지면, 기존에 발행된 채권의 가치가 하락하게 되기 때문입니다. 따라서 채권 투자자들은 높은 이자율을 받기 위해 새로운 채권을 구매하게 되고, 기존에 보유하고 있던 채권을 팔게 되어 채권가격이 하락하게 됩니다.'

## 6. 수학문제에 잘 답변해주도록 만드는 LLMMathChain

In [129]:
from langchain import OpenAI, LLMMathChain
llm = OpenAI(temperature=0)
llm_math = LLMMathChain.from_llm(llm, verbose=True)
llm_math.run("13.87 ** 33.5?")



> Entering new LLMMathChain chain...
13.87 ** 33.5?```text
13.87 ** 33.5
```
...numexpr.evaluate("13.87 ** 33.5")...

Answer: 1.8178642375870954e+38
> Finished chain.


'Answer: 1.8178642375870954e+38'

In [147]:
print(llm_math.llm_chain.prompt.template) # 퓨샷러닝 방식으로 수학문제를 잘 답변하게 만들어줌

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



## 7. 환각을 줄여주는 LLMCheckerChain
> LLM이 팩트를 체크하고, 근거에 따라 확인하는 과정을 거친다.

사실 확인은 세가지 단계를 거친다. 
1. 주장감지 : 검증 필요한 부분 식별
2. 증거검색 : 주장 지지하거나 반박소스 찾기
3. 판결예측 : 증거 기반으로 진위평가

In [134]:
from langchain.chains import LLMCheckerChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

text = "한국에서 제일 부자는? 이유도 설명해줘"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

checker_chain.run(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


' 한국에서 제일 부자는 삼성그룹의 회장인 이재용입니다. 그는 삼성그룹의 다양한 산업에서 성공적인 사업을 운영하고 국내외에서 상당한 이익을 창출하며, 삼성그룹은 한국 경제에서 주요한 역할을 합니다. 이재용은 삼성그룹 회장의 지위로 인해 부를 누리고 있지만, 그는 자신의 사업과 투자를 통해 상당한 부를 축적했습니다. 또한 삼성그룹은 한국에서 주요한 고용주이기도 합니다. 하지만 이러한 상황은 현재의 정보를 기반으로 한 것이므로, 미래에는 변할 수 있습니다.'

- checker_chain을 확인해보면, 질문에 대해 2개이상의 LLM chat에 답변을 얻고, 다음과 같은 프롬프트를 생성하여 검증한다<br>
template=<br>
'Here is a bullet point list of assertions:<br>
{assertions}<br>
For each assertion, determine whether it is true or false. If it is false, explain why.\n\n'
<br><br>
주장의 핵심 목록은 다음과 같음:<br>
{주장}<br>
각 주장에 대해, 참인지 거진인지 판단하고, 거짓이면 이유를 밝혀라.<br>